In [4]:

#Importing Libraries
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')
import re
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [91]:
zomato_real=pd.read_csv('zomato 3.csv')


In [92]:
zomato_real

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51712,https://www.zomato.com/bangalore/best-brews-fo...,"Four Points by Sheraton Bengaluru, 43/3, White...",Best Brews - Four Points by Sheraton Bengaluru...,No,No,3.6 /5,27,080 40301477,Whitefield,Bar,NaN,Continental,"1,500","[('Rated 5.0', ""RATED\n Food and service are ...",[],Pubs and bars,Whitefield
51713,https://www.zomato.com/bangalore/vinod-bar-and...,"Number 10, Garudachar Palya, Mahadevapura, Whi...",Vinod Bar And Restaurant,No,No,NaN,0,+91 8197675843,Whitefield,Bar,NaN,Finger Food,600,[],[],Pubs and bars,Whitefield
51714,https://www.zomato.com/bangalore/plunge-sherat...,Sheraton Grand Bengaluru Whitefield Hotel & Co...,Plunge - Sheraton Grand Bengaluru Whitefield H...,No,No,NaN,0,NaN,Whitefield,Bar,NaN,Finger Food,"2,000",[],[],Pubs and bars,Whitefield
51715,https://www.zomato.com/bangalore/chime-sherato...,Sheraton Grand Bengaluru Whitefield Hotel & Co...,Chime - Sheraton Grand Bengaluru Whitefield Ho...,No,Yes,4.3 /5,236,080 49652769,"ITPL Main Road, Whitefield",Bar,"Cocktails, Pizza, Buttermilk",Finger Food,"2,500","[('Rated 4.0', 'RATED\n Nice and friendly pla...",[],Pubs and bars,Whitefield


In [93]:
zomato=zomato_real.drop(['url','dish_liked','phone'],axis=1) #Dropping the column "dish_liked", "phone", "url" and saving the new dataset as "zomato"



In [94]:
#Removing the Duplicates
zomato.duplicated().sum()
zomato.drop_duplicates(inplace=True)



In [95]:
#Remove the NaN values from the dataset
zomato.isnull().sum()
zomato.dropna(how='any',inplace=True)



In [96]:
#Changing the column names
zomato = zomato.rename(columns={'approx_cost(for two people)':'cost','listed_in(type)':'type', 'listed_in(city)':'city'})



In [97]:
#Some Transformations
zomato['cost'] = zomato['cost'].astype(str) #Changing the cost to string
zomato['cost'] = zomato['cost'].apply(lambda x: x.replace(',','.')) #Using lambda function to replace ',' from cost
zomato['cost'] = zomato['cost'].astype(float)


In [99]:
# Adjust the column names
zomato.name = zomato.name.apply(lambda x:x.title())
zomato.online_order.replace(('Yes','No'),(True, False),inplace=True)
zomato.book_table.replace(('Yes','No'),(True, False),inplace=True)



In [102]:
# Extract numeric part of the 'rate' column and convert to float
zomato['rate'] = zomato['rate'].str.extract(r'(\d+\.?\d*)')[0]  # Extract numbers
zomato['rate'] = pd.to_numeric(zomato['rate'], errors='coerce')  # Convert to float


In [103]:
zomato['rate'].fillna(zomato['rate'].mean(), inplace=True)


In [104]:
restaurants = list(zomato['name'].unique())
zomato['Mean Rating'] = 0.0

for i in range(len(restaurants)):
    zomato.loc[zomato['name'] == restaurants[i], 'Mean Rating'] = zomato.loc[zomato['name'] == restaurants[i], 'rate'].mean()


In [108]:
## Lower Casing
zomato["reviews_list"] = zomato["reviews_list"].str.lower()



In [109]:
## Removal of Puctuations
import string
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

zomato["reviews_list"] = zomato["reviews_list"].apply(lambda text: remove_punctuation(text))



In [110]:
## Removal of Stopwords
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

zomato["reviews_list"] = zomato["reviews_list"].apply(lambda text: remove_stopwords(text))



In [111]:
## Removal of URLS
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

zomato["reviews_list"] = zomato["reviews_list"].apply(lambda text: remove_urls(text))

zomato[['reviews_list', 'cuisines']].sample(5)

,reviews_list,cuisines
14476,rated 20 ratedn food ambiance good upto expect...,"Lebanese, Arabian, Middle Eastern, Salad, Beve..."
39111,rated 10 ratedn hopeless cold soggy chicken le...,Fast Food
8294,rated 50 ratedn love place go even cold soupng...,"Chinese, Momos"
27231,,"Chinese, Fast Food"
12872,rated 50 ratedn good food value money im glad ...,Biryani


In [112]:
# RESTAURANT NAMES:
restaurant_names = list(zomato['name'].unique())
def get_top_words(column, top_nu_of_words, nu_of_word):
    vec = CountVectorizer(ngram_range= nu_of_word, stop_words='english')
    bag_of_words = vec.fit_transform(column)
    sum_words = bag_of_words.sum(axis=0)
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:top_nu_of_words]
    
zomato=zomato.drop(['address','rest_type', 'type', 'menu_item', 'votes'],axis=1)
import pandas

# Randomly sample 60% of your dataframe
df_percent = zomato.sample(frac=0.5)

In [115]:
# Creating tf-idf matrix
tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=1, stop_words='english')
tfidf_matrix = tfidf.fit_transform(df_percent['reviews_list'])

# Compute cosine similarities
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

print("TF-IDF matrix shape:", tfidf_matrix.shape)

TF-IDF matrix shape: (20618, 1359272)


In [116]:
df_percent.head()

,online_order,book_table,rate,location,cuisines,cost,reviews_list,city,Mean Rating
name,,,,,,,,,
Savoury - Sea Shell Restaurant,True,False,3.9,BTM,"Arabian, North Indian, Chinese, Fast Food",700.0,rated 10 ratedn great reasturant taste wise fo...,Koramangala 7th Block,3.900000
Durga Sagar,True,False,3.4,Shanti Nagar,South Indian,150.0,rated 40 ratedn ordered anna sambar given sepa...,Koramangala 7th Block,3.363636
White Horse,False,False,3.8,JP Nagar,"North Indian, Mangalorean, Chinese",400.0,rated 50 ratedn 2 years back chicken biriyani ...,JP Nagar,3.733333
Samrat Regency Bar And Restaurant,False,False,3.8,Koramangala 6th Block,"North Indian, Chinese",800.0,rated 50 ratedn bar quiet gud fr pork staters ...,Koramangala 5th Block,3.800000
Bangalore Durbar Fast Food,True,False,3.6,Thippasandra,"North Indian, South Indian, Chinese, Kerala",600.0,rated 10 ratedn ãx83ãx83ãx82ãx83ãx83ãx82...,Old Airport Road,3.600000


In [117]:
print(df_percent['reviews_list'].head())
print(df_percent['reviews_list'].isnull().sum())


name
Savoury - Sea Shell Restaurant       rated 10 ratedn great reasturant taste wise fo...
Durga Sagar                          rated 40 ratedn ordered anna sambar given sepa...
White Horse                          rated 50 ratedn 2 years back chicken biriyani ...
Samrat Regency Bar And Restaurant    rated 50 ratedn bar quiet gud fr pork staters ...
Bangalore Durbar Fast Food           rated 10 ratedn ãx83ãx83ãx82ãx83ãx83ãx82...
Name: reviews_list, dtype: object
0


In [118]:
print(cosine_similarities[:5, :5])


[[1.00000000e+00 1.14774654e-02 1.31497869e-02 3.09620226e-02
  2.86546357e-04]
 [1.14774654e-02 1.00000000e+00 1.85784369e-02 2.05874168e-02
  1.95123519e-04]
 [1.31497869e-02 1.85784369e-02 1.00000000e+00 2.80391503e-02
  2.65178473e-04]
 [3.09620226e-02 2.05874168e-02 2.80391503e-02 1.00000000e+00
  2.49253894e-04]
 [2.86546357e-04 1.95123519e-04 2.65178473e-04 2.49253894e-04
  1.00000000e+00]]


In [119]:
print(indices.head())


0       Savoury - Sea Shell Restaurant
1                          Durga Sagar
2                          White Horse
3    Samrat Regency Bar And Restaurant
4           Bangalore Durbar Fast Food
Name: name, dtype: object


In [131]:
def recommend(name, cosine_similarities=cosine_similarities):
    recommend_restaurant = []
    idx = indices[indices == name].index[0]   
    score_series = pd.Series(cosine_similarities[idx]).sort_values(ascending=False)    
    top30_indexes = list(score_series.iloc[1:31].index)     
    recommend_restaurant = [list(df_percent.index)[each] for each in top30_indexes]   
    rows = []
    for each in recommend_restaurant:
        
        row = df_percent.loc[each, ['cuisines', 'Mean Rating', 'cost']]
        if isinstance(row, pd.Series):
            rows.append(row.to_dict())
        elif isinstance(row, pd.DataFrame):
            rows.append(row.iloc[0].to_dict())

    df_new = pd.DataFrame(rows)
    df_new = df_new.drop_duplicates(subset=['cuisines', 'Mean Rating', 'cost'], keep='first')
    df_new = df_new.sort_values(by='Mean Rating', ascending=False).head(10)   
    print(f'TOP {len(df_new)} RESTAURANTS LIKE {name} WITH SIMILAR REVIEWS:')   
    return df_new

  



In [136]:
recommendations = recommend('Durga Sagar')
print(recommendations)


TOP 10 RESTAURANTS LIKE Durga Sagar WITH SIMILAR REVIEWS:
                                            cuisines  Mean Rating   cost
17                    North Indian, Mughlai, Biryani     3.931579  400.0
11                                Cafe, South Indian     3.800000  200.0
6                           Biryani, Chinese, Andhra     3.800000  500.0
18                    North Indian, Chinese, Biryani     3.650000  400.0
23               North Indian, Chinese, Kebab, Rolls     3.600000  300.0
27                    North Indian, Chinese, Biryani     3.600000  450.0
25                         South Indian, Street Food     3.600000  150.0
8   South Indian, North Indian, Chinese, Street Food     3.551429  300.0
15                                  Mughlai, Biryani     3.505882  400.0
10                             North Indian, Chinese     3.500000  400.0
